In [ ]:
import pandas as pd

# Especifica la ruta de tu archivo CSV
ruta_del_archivo = 'violence_dataset.csv'

# Lee el archivo CSV en un DataFrame
df = pd.read_csv(ruta_del_archivo)

df.head()

In [ ]:
!pip install -U sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer, util

In [ ]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
embeddings = model.encode(df['examples'],batch_size=64,show_progress_bar=True)

In [ ]:
df['embeddings'] = embeddings.tolist()

In [ ]:
df['ids'] = df.index
df['ids'] = df['ids'].astype('str')

In [ ]:
df

In [ ]:
df.to_csv('embeddings.csv', index=False)

# BD

In [ ]:
!pip install chromadb

In [ ]:
import chromadb
from chromadb.utils import embedding_functions
import pandas as pd

In [ ]:
df = pd.read_csv('embeddings.csv')

In [ ]:
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name = 'all-MiniLM-L6-v2')

In [ ]:
chroma_client = chromadb.Client()
client_persistent = chromadb.PersistentClient(path='/content/data_embeddings')

In [ ]:
db = client_persistent.create_collection(name='violence_exampless_db', embedding_function=sentence_transformer_ef)

In [ ]:
df['ids'] = df['ids'].astype(str)

In [ ]:
import ast

In [ ]:
# Convierte las cadenas en listas de números en la columna 'embeddings'
df['embeddings'] = df['embeddings'].apply(lambda x: ast.literal_eval(x))

In [ ]:
db.add(
    ids=df['ids'].tolist(),
    embeddings=df['embeddings'].tolist(),
    metadatas= df.drop(['ids','embeddings','examples'],axis=1).to_dict('records')
)

In [ ]:
db.peek(1)

# Chroma embeddings

In [ ]:
db_chroma_embeddings = client_persistent.create_collection(name='violence_chroma_db', embedding_function= sentence_transformer_ef)

In [ ]:
db_chroma_embeddings.add(
    ids=df['ids'].tolist(),
    documents=df['examples'].tolist(),
    metadatas= df.drop(['ids','embeddings','examples'],axis=1).to_dict('records')
)

In [ ]:
db_chroma_embeddings.peek(1)

# query

In [ ]:
results = db_chroma_embeddings.query(
    query_texts=['estoy muy feliz'],
    n_results=2
)

In [ ]:
results

# TSV

In [13]:
import pandas as pd

df = pd.read_csv('embeddings.csv')

new_df = pd.DataFrame(df['embeddings'].str.strip('[]').str.split(',').tolist()).astype(float)
new_df.columns = range(0, new_df.shape[1] )

print(new_df)


          0         1         2         3         4         5         6    \
0   -0.104456  0.084571 -0.027690 -0.011262 -0.064720 -0.037093  0.062168   
1   -0.040397  0.036325 -0.037555  0.030470 -0.076000  0.097781  0.082546   
2   -0.050412  0.083849 -0.077063 -0.013987 -0.138557  0.019924  0.042121   
3   -0.037046 -0.024938 -0.018664  0.078173 -0.095013 -0.063062  0.107255   
4   -0.029419  0.070571 -0.044096 -0.053635 -0.022133  0.015794  0.099863   
..        ...       ...       ...       ...       ...       ...       ...   
226 -0.110963  0.006407  0.021113 -0.013736 -0.090332  0.042317  0.069285   
227 -0.061404  0.019444 -0.014598  0.044330 -0.069735  0.043872  0.048477   
228 -0.064904  0.012356 -0.035034  0.054927 -0.011320  0.055607  0.018376   
229 -0.002704  0.010834 -0.054450  0.081948 -0.079543  0.023822  0.079851   
230  0.028132  0.014874 -0.099816  0.041598 -0.038118  0.021323  0.101052   

          7         8         9    ...       374       375       376  \
0  

In [14]:
new_df.to_csv('embeddings.tsv', sep='\t', index=False,header=False)

In [23]:
import pandas as pd

df = pd.read_csv('violence_dataset.csv')

print(df)

     class             examples
0        0                 orto
1        0                bulto
2        0               cajeta
3        0                choto
4        0                coger
..     ...                  ...
226      1   te vas a la mierda
227      1   maten a los negros
228      1  maten a los blancos
229      1     haré una masacre
230      1            maricones

[231 rows x 2 columns]


In [24]:
df.to_csv('violence_labels.tsv', sep='\t', index=False,header=True)